In [ ]:
from collections import defaultdict
from typing import Sequence
import pandas as pd

SUBMISSION_PATHS = ['MODEL1/PATH/submission.csv',
                    'MODEL2/PATH/submission.csv',
                    'MODEL3/PATH/submission.csv',
                    'MODEL4/PATH/submission.csv',
                   ]

MERGED_SUBMISSION_PATH = 'MERGED/PATH/submission.csv',

In [ ]:
def merge_submissions(submissions: Sequence[pd.DataFrame]) -> pd.DataFrame:
    sub_dict = defaultdict(lambda: [])
    merged_sub = defaultdict(lambda: [])
    for sub in submissions:
        for _, row in sub.iterrows():
            pred_string = str(row['PredictionString'])
            sub_dict[row['ImageId']]
            if 'nan' not in pred_string:
                sub_dict[row['ImageId']].append(pred_string)
    for key, value in sub_dict.items():
        merged_sub['ImageId'].append(key)
        merged_sub['PredictionString'].append(' '.join(value))
    final_sub = pd.DataFrame.from_dict(merged_sub)
    return final_sub

In [ ]:
subs = []
for submission_path in SUBMISSION_PATHS:
    batch_submission = pd.read_csv(SUBMISSION_PATHS)
    subs.append(batch_submission)

In [ ]:
submission = merge_submissions(subs)
submission.head()

In [ ]:
submission.to_csv(MERGED_SUBMISSION_PATH, index=False)